In [1]:
import pandas as pd

In [7]:
billboard_charts = pd.read_csv(r'C:\Users\Xonas\Documents\UFPB\IA-MUSIC\data\billboard_hot100_charts.csv', sep="\t")
billboard_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323787 entries, 0 to 323786
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   track           323787 non-null  object
 1   artist          323787 non-null  object
 2   peak_pos        323787 non-null  int64 
 3   last_week       323787 non-null  int64 
 4   weeks_on_chart  323787 non-null  int64 
 5   chart_pos       323787 non-null  int64 
 6   isNew           323787 non-null  bool  
 7   chart_week      323787 non-null  object
 8   chart           323787 non-null  object
dtypes: bool(1), int64(4), object(4)
memory usage: 20.1+ MB


In [8]:
billboard_spotify_charts = pd.read_csv(r'C:\Users\Xonas\Documents\UFPB\IA-MUSIC\data\billboard_hot100_songs_to_spotify_with_features.csv', sep="\t")
billboard_spotify_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24540 entries, 0 to 24539
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_id           24540 non-null  object 
 1   song_name         24540 non-null  object 
 2   billboard         24540 non-null  object 
 3   artists           24540 non-null  object 
 4   popularity        24540 non-null  int64  
 5   explicit          24540 non-null  bool   
 6   song_type         24540 non-null  object 
 7   track_number      24540 non-null  int64  
 8   num_artists       24540 non-null  int64  
 9   danceability      24540 non-null  float64
 10  energy            24540 non-null  float64
 11  key               24540 non-null  float64
 12  loudness          24540 non-null  float64
 13  mode              24540 non-null  float64
 14  speechiness       24540 non-null  float64
 15  acousticness      24540 non-null  float64
 16  instrumentalness  24540 non-null  float6

In [ ]:
sucess = billboard_charts[billboard_charts['chart_pos']<=5].drop_duplicates(subset='track')
failed = billboard_charts[billboard_charts['track']]

,track,artist,peak_pos,last_week,weeks_on_chart,chart_pos,isNew,chart_week,chart
0,WAP,Cardi B Featuring Megan Thee Stallion,1,0,1,1,True,2020-08-22,hot-100
100,Watermelon Sugar,Harry Styles,1,7,20,1,False,2020-08-15,hot-100
200,Cardigan,Taylor Swift,1,0,1,1,True,2020-08-08,hot-100
300,Rockstar,DaBaby Featuring Roddy Ricch,1,1,14,1,False,2020-08-01,hot-100
400,Rockstar,DaBaby Featuring Roddy Ricch,1,1,13,1,False,2020-07-25,hot-100
...,...,...,...,...,...,...,...,...,...
323287,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1,1,6,1,False,1958-09-08,hot-100
323387,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1,2,5,1,False,1958-09-01,hot-100
323487,Little Star,The Elegants,1,2,4,1,False,1958-08-25,hot-100
323587,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1,2,3,1,False,1958-08-18,hot-100


In [ ]:
import pandas as pd

# Dados de exemplo
data1 = {'ID': [1, 2, 3, 4, 5], 'Valor': ['A', 'B', 'C', 'D', 'E']}
data2 = {'ID': [1, 3, 5], 'OutroValor': ['X', 'Y', 'Z']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Combinar os DataFrames
df_merged = pd.merge(df1, df2, on='ID', how='left')

# Verificar se as linhas são duplicadas (na segunda tabela)
df_merged['Duplicado'] = df_merged['OutroValor'].notna()

# Remover linhas duplicadas, mantendo as que não existem na segunda tabela
df_filtered = df_merged[~df_merged['Duplicado']]

# Remover a coluna auxiliar
df_filtered = df_filtered.drop(columns=['OutroValor', 'Duplicado'])

# Exibir o resultado
print(df_filtered)